In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [3]:
# tst[tst['id']=='HOBBIES_1_210_CA_1_validation']

In [4]:
# tst = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
# np.where(tst.iloc[0,6:].values>0)[0].max()
# tst.iloc[0,6:].values>0
# tst[tst['id']=='HOBBIES_1_210_CA_1_validation']
# for i, val in last_dict.items():
#     if val<1750 : print(i)

In [5]:
# # startpoints 찾아서 이전 데이터 지우기 + last sales를 찾기
startpoints = np.zeros(df_train.shape[0])
lastpoints = np.zeros(df_train.shape[0])
for idx in tqdm(range(df_train.shape[0])):
    startpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].min().astype(int)
    lastpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].max().astype(int)
start_dict = dict(zip(df_train['id'], startpoints))
last_dict = dict(zip(df_train['id'], lastpoints))

100%|███████████████████████████████████████████████████████████████████████████| 30490/30490 [01:13<00:00, 414.35it/s]


In [6]:
drop_cols = []
cat_cols = []
drop_cols += ['date','d','id']
tr_last = 1913
# F_1~28 만들기  1914~1941 까지 
for i in range(tr_last+1, tr_last+1+28):   df_train['d_%s'%i] = 0

# # Unpivot
df_train = pd.melt(df_train, id_vars=df_train.columns[:6], value_vars=df_train.columns[6:],
       var_name = 'day', value_name = 'volume')



In [7]:
df_train = pd.merge(df_train, calendar, left_on = 'day', right_on ='d')
# snap 합치기
snap = np.zeros(df_train.shape[0])
snap[df_train[(df_train['state_id']=='CA')&(df_train['snap_CA']==1)].index] +=1
snap[df_train[(df_train['state_id']=='TX')&(df_train['snap_TX']==1)].index] +=1
snap[df_train[(df_train['state_id']=='WI')&(df_train['snap_WI']==1)].index] +=1
df_train['snap'] = snap
drop_cols += ['snap_CA','snap_TX','snap_WI']


cat_cols += [ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
#               'wday', 'month', 'year', # 이게 크리티컬?
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap']


# ????

In [8]:
# Sell price
df_train.head()
df_train = pd.merge(df_train, price)

# # Start point 찾기 ::  0.1% 데이터를 날릴  수 있다. 

df_train['startpoint'] = df_train['id'].map(start_dict).astype(int)#.astype(str)
df_train['startpoints'] = df_train['day'].str.slice(start=2).astype(int) >=df_train['startpoint']
print(df_train['startpoints'].value_counts())
df_train = df_train[df_train['startpoints']]
print(df_train.shape)
df_train.drop(['startpoint','startpoints'],axis =1, inplace= True)

True     46816555
False       65122
Name: startpoints, dtype: int64
(46816555, 26)


In [9]:
df_train['lastpoint'] = df_train['id'].map(last_dict).astype(int)#.astype(str)
df_train['from_lastpoint'] = df_train['day'].str.slice(start=2).astype(int) - df_train['lastpoint']

df_train.drop(['lastpoint'],axis =1, inplace= True)

In [10]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,volume,date,wm_yr_wk,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap,sell_price,from_lastpoint
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1911
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1910
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1909
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,1,1,0,1.0,0.46,-1908
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,1,0,1,1.0,0.46,-1907


In [11]:
df_train

,id,item_id,dept_id,cat_id,store_id,state_id,day,volume,date,wm_yr_wk,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap,sell_price,from_lastpoint
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1911
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1910
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,0,0,0,0.0,0.46,-1909
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,1,1,0,1.0,0.46,-1908
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,1,0,1,1.0,0.46,-1907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,3.98,30
46881673,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1940,0,2016-05-21,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.28,28
46881674,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.28,29
46881675,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1940,0,2016-05-21,11617,...,NaN,NaN,NaN,NaN,0,0,0,0.0,1.00,43


In [12]:
# df_train['rcount_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# df_train[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0 else 0 )#.fllna(0)


In [13]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

# df_train['volume_1'] = df_train[['id','volume']].groupby("id")['volume'].shift(1)
# df_train['volume_2'] = df_train[['id','volume']].groupby("id")['volume'].shift(2)
# df_train['volume_3'] = df_train[['id','volume']].groupby("id")['volume'].shift(3)

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())

print("std")
df_train['rstd_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).std())
df_train['rstd_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).std())
df_train['rstd_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).std())

df_train['rstd_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).std())
df_train['rstd_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).std())
df_train['rstd_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).std())

print("max")
df_train['rmax_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).max())
df_train['rmax_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).max())
df_train['rmax_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).max())

df_train['rmax_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).max())
df_train['rmax_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).max())
df_train['rmax_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).max())

print("min")
df_train['rmin_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).min())
df_train['rmin_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).min())
df_train['rmin_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).min())

df_train['rmin_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).min())
df_train['rmin_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).min())
df_train['rmin_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).min())

# print("count")
# df_train['rcount_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# df_train['rcount_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# df_train['rcount_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

# df_train['rcount_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).count() if x>0 else 0).fllna(0)
# df_train['rcount_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).count() if x>0 else 0).fllna(0)
# df_train['rcount_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).count() if x>0 else 0).fllna(0)

# full_df['rcnt_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# full_df['rcnt_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# full_df['rcnt_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)


mean
std
max
min
Wall time: 13min 13s


In [14]:
df_train['date'] =  pd.to_datetime(df_train["date"])

In [15]:
df_train['week'] = getattr(df_train["date"].dt, "weekofyear").astype("int16")
# df_train['quarter'] = getattr(df_train["date"].dt,"quarter").astype("int16")
df_train['mday'] = getattr(df_train["date"].dt, "day").astype("int16")


In [16]:
cols =['event_name_1','event_type_1','event_name_2','event_type_2']
df_train[cols]= df_train[cols].fillna('NaN')
print(df_train.shape)


(46816555, 56)


In [34]:
gc.collect()

168

In [31]:
df_train.shape

(46816555, 56)

In [32]:
df_train = df_train[~df_train['rmean_28_50'].isna()]
# dropna 하면 뻥난다.

In [35]:
for col in df_train.columns:
    print(col, df_train[col].isnull().sum())
# df_train.isnull().sum()

id 0
item_id 0
dept_id 0
cat_id 0
store_id 0
state_id 0
day 0
volume 0
date 0
wm_yr_wk 0
weekday 0
wday 0
month 0
year 0
d 0
event_name_1 0
event_type_1 0
event_name_2 0
event_type_2 0
snap_CA 0
snap_TX 0
snap_WI 0
snap 0
sell_price 0
from_lastpoint 0
volume_1 0
volume_2 0
volume_3 0
volume_7 0
volume_28 0
rmean_7_7 0
rmean_7_28 0
rmean_7_50 0
rmean_28_7 0
rmean_28_28 0
rmean_28_50 0
rstd_7_7 0
rstd_7_28 0
rstd_7_50 0
rstd_28_7 0
rstd_28_28 0
rstd_28_50 0
rmax_7_7 0
rmax_7_28 0
rmax_7_50 0
rmax_28_7 0
rmax_28_28 0
rmax_28_50 0
rmin_7_7 0
rmin_7_28 0
rmin_7_50 0
rmin_28_7 0
rmin_28_28 0
rmin_28_50 0
week 0
mday 0


In [36]:
# df_train.dropna()#inplace =True

In [37]:
df_train.shape

(44468825, 56)

In [38]:
drop_cols += ["wm_yr_wk", "weekday"]  ## 이게 문제?
drop_cols

['date', 'd', 'id', 'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk', 'weekday']

In [39]:
df_train.head()
tr_last = 1913
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
train_id = df_train['id']
df_test_id = df_train[df_train['day'].isin(testday)]['id']

In [40]:
df_train= df_train.drop(drop_cols,axis =1 )
# df_test =df_test.drop(drop_cols,axis =1)

In [41]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,rmax_28_28,rmax_28_50,rmin_7_7,rmin_7_28,rmin_7_50,rmin_28_7,rmin_28_28,rmin_28_50,week,mday
1010975,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_78,0,1,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,16
1010976,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_79,0,2,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,17
1010977,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_80,0,3,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,18
1010978,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_81,5,4,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,19
1010979,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_82,23,5,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,20


In [42]:
# Encoding
for col in tqdm(cat_cols) :  # encoding -1이 문제?
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col]).astype(np.int8)

    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:10<00:00,  7.00s/it]


In [43]:
# Test 분리하기
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
df_test = df_train.copy()
df_train = df_train[~df_train['day'].isin(testday)]
# train_col = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day', 'volume', 'wday', 'month', 'year',
#  'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap', 'sell_price', 'volume_7', 'volume_28',
#  'rmean_7_7', 'rmean_7_28', 'rmean_7_50', 'rmean_28_7', 'rmean_28_28', 'rmean_28_50', 'week', 'quarter', 'mday']
# df_train = df_train.loc[:,train_col]

In [44]:
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,rmax_28_28,rmax_28_50,rmin_7_7,rmin_7_28,rmin_7_50,rmin_28_7,rmin_28_28,rmin_28_50,week,mday
1010975,-92,3,1,0,0,d_78,0,1,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,16
1010976,-92,3,1,0,0,d_79,0,2,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,17
1010977,-92,3,1,0,0,d_80,0,3,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,18
1010978,-92,3,1,0,0,d_81,5,4,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,19
1010979,-92,3,1,0,0,d_82,23,5,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,20


In [45]:
# %%time

# from sklearn.model_selection import train_test_split

# x_train , x_valid = train_test_split(df_train, test_size =0.15, random_state = 99)
# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

In [46]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,rmax_28_28,rmax_28_50,rmin_7_7,rmin_7_28,rmin_7_50,rmin_28_7,rmin_28_28,rmin_28_50,week,mday
1010975,-92,3,1,0,0,d_78,0,1,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,16
1010976,-92,3,1,0,0,d_79,0,2,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,17
1010977,-92,3,1,0,0,d_80,0,3,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,18
1010978,-92,3,1,0,0,d_81,5,4,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,19
1010979,-92,3,1,0,0,d_82,23,5,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,20


In [47]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 10000,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
        'n_jobs' :10 
}

In [48]:
%%time
seed = 99
folds = 3
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error
import pickle

skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)

for idx, (train_index, test_index) in enumerate(skf.split(df_train.index,df_train['volume'])):
    print(idx)
    x_train = df_train.iloc[train_index].drop(['day','volume'], axis =1)
    y_train = df_train.iloc[train_index]['volume']
    x_valid = df_train.iloc[test_index].drop(['day','volume'], axis =1)
    y_valid = df_train.iloc[test_index]['volume'] 
    
    # Modeling
    lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
    lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
    gbm = lgb.train(params, lgb_train,
#                     num_boost_round=1000, 
                    valid_sets=(lgb_train, lgb_eval),
                    early_stopping_rounds= 50,#100,
                    verbose_eval=200) #100)

    pickle.dump(gbm,open( "20200501_model_%s_r2.pkl"%idx, "wb" ))
        
    del gbm
    gc.collect()
    

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[200]	training's rmse: 2.25134	valid_1's rmse: 2.30223
[400]	training's rmse: 2.17929	valid_1's rmse: 2.27837
[600]	training's rmse: 2.13866	valid_1's rmse: 2.269
Early stopping, best iteration is:
[618]	training's rmse: 2.13591	valid_1's rmse: 2.26865
1
Training until validation scores don't improve for 50 rounds
[200]	training's rmse: 2.25907	valid_1's rmse: 2.31285
[400]	training's rmse: 2.18477	valid_1's rmse: 2.27329
[600]	training's rmse: 2.14307	valid_1's rmse: 2.26023
[800]	training's rmse: 2.115	valid_1's rmse: 2.25519
[1000]	training's rmse: 2.09242	valid_1's rmse: 2.25177
[1200]	training's rmse: 2.07171	valid_1's rmse: 2.24979
[1400]	training's rmse: 2.05558	valid_1's rmse: 2.24793
[1600]	training's rmse: 2.04121	valid_1's rmse: 2.24668
[1800]	training's rmse: 2.02843	valid_1's rmse: 2.24546
[2000]	training's rmse: 2.01626	valid_1's rmse: 2.24443
[2200]	training's rmse: 2.00558	valid_1's rmse: 2.24363
[2400]	traini

In [52]:
%%time
df_test['id'] = train_id
max_lag  = 120
trn_lst = 1913
for tdelta in range(1, 29):
    f_day = trn_lst+tdelta
    print(f_day)
    days = [f"d_{i}" for i in range(trn_lst-max_lag+tdelta,f_day+1)]
    tst = df_test[df_test['day'].isin(days)]

    print("rolling")
#     tst['volume_1'] = tst[['id','volume']].groupby("id")['volume'].shift(1)
#     tst['volume_2'] = tst[['id','volume']].groupby("id")['volume'].shift(2)
#     tst['volume_3'] = tst[['id','volume']].groupby("id")['volume'].shift(3)
    
    tst['volume_7'] = tst[['id','volume']].groupby("id")['volume'].shift(7)
    tst['volume_28'] = tst[['id','volume']].groupby("id")['volume'].shift(28)
    
    print("mean")
    tst['rmean_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
    tst['rmean_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

    tst['rmean_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
    tst['rmean_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())
    
    print("std")
    tst['rstd_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).std())
    tst['rstd_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).std())
    tst['rstd_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).std())

    tst['rstd_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).std())
    tst['rstd_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).std())
    tst['rstd_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).std())

    print("max")
    tst['rmax_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).max())
    tst['rmax_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).max())
    tst['rmax_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).max())

    tst['rmax_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).max())
    tst['rmax_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).max())
    tst['rmax_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).max())

    print("min")
    tst['rmin_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).min())
    tst['rmin_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).min())
    tst['rmin_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).min())

    tst['rmin_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).min())
    tst['rmin_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).min())
    tst['rmin_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).min())

#     print("count")
#     tst['rcount_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
#     tst['rcount_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
#     tst['rcount_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

#     tst['rcount_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
#     tst['rcount_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
#     tst['rcount_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).count() if x>0).fllna(0)

    
    tst = tst[tst['day'] == "d_%s"%(f_day)]
    t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
    tst = tst.drop(['id','volume','day'], axis =1)
    
    # Crossvalidation 
    for idx in range(folds):
        gbm = pickle.load(open( "20200501_model_%s_r2.pkl"%idx, "rb" ))
        df_test.loc[df_test.day=="d_%s"%(f_day),'volume'] += 1.028*gbm.predict(tst) / folds
        del gbm
        gc.collect()
 
    del tst 
    gc.collect()
    #0을 예측 못하니까 0으로 치환하는 것 -- test
#     df_test.loc[df_test.volume<0.5,'volume'] = 0
    
    


1914
rolling


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

mean


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

std


...

max


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

min


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

1915
rolling
mean
std
max
min
1916
rolling
mean
std
max
min
1917
rolling
mean
std
max
min
1918
rolling
mean
std
max
min
1919
rolling
mean
std
max
min
1920
rolling
mean
std
max
min
1921
rolling
mean
std
max
min
1922
rolling
mean
std
max
min
1923
rolling
mean
std
max
min
1924
rolling
mean
std
max
min
1925
rolling
mean
std
max
min
1926
rolling
mean
std
max
min
1927
rolling
mean
std
max
min
1928
rolling
mean
std
max
min
1929
rolling
mean
std
max
min
1930
rolling
mean
std
max
min
1931
rolling
mean
std
max
min
1932
rolling
mean
std
max
min
1933
rolling
mean
std
max
min
1934
rolling
mean
std
max
min
1935
rolling
mean
std
max
min
1936
rolling
mean
std
max
min
1937
rolling
mean
std
max
min
1938
rolling
mean
std
max
min
1939
rolling
mean
std
max
min
1940
rolling
mean
std
max
min
1941
rolling
mean
std
max
min
Wall time: 2h 26min 11s


In [53]:
cols = [f"d_{i}" for i in range(1914,1942)]

sub = df_test[df_test['day'].isin(cols)].loc[:,['id','volume']]
sub['F']= [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
sub = sub.set_index(["id", "F" ]).unstack()["volume"].reset_index()
sub.sort_values("id", inplace = True)
sub.reset_index(drop=True, inplace = True)                                                   
sub =sub[['id']+["F%s"% x for x in range(1,29)]]

sub = sub.fillna(0)

sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_20200501_1.csv",index=False)

In [59]:
gc.collect()

103

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,rmax_28_28,rmax_28_50,rmin_7_7,rmin_7_28,rmin_7_50,rmin_28_7,rmin_28_28,rmin_28_50,week,mday
1010975,-92,3,1,0,0,d_78,0,1,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,16
1010976,-92,3,1,0,0,d_79,0,2,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,15,17
1010977,-92,3,1,0,0,d_80,0,3,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,18
1010978,-92,3,1,0,0,d_81,5,4,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,19
1010979,-92,3,1,0,0,d_82,23,5,4,2011,...,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,16,20


In [62]:
# Feature 체크하기
import pickle
gbm = pickle.load(open( "20200501_model_2_r2.pkl", "rb" ))
# %%time

from sklearn.model_selection import train_test_split

def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))


# x_train , x_valid = train_test_split(df_train, test_size =0.05, random_state = 99)
# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

cols = ["d_%s"% x for x in range(1883,1914)]
x_valid = df_train[df_train['day'].isin(cols)]
y_valid = x_valid['volume']
x_valid = x_valid.drop(['day','volume'], axis =1)

In [63]:
%%time
pred = gbm.predict(x_valid)
print("original", rmse(y_valid, pred))
origin = rmse(y_valid, pred)

original 1.8499813687516031
Wall time: 26.5 s


In [ ]:
%%time 
# 컬럼들을 랜덤하게 섞어서 예측해보는 것
# cols = list(x_valid.columns)

z = pd.read_csv('featuretest.csv')
cols = list(x for x in x_valid.columns if x not in list(z['columns']))
fin = dict()
for col in tqdm(cols):
    tmp = x_valid.copy()
    tmp[col] = np.random.permutation(tmp[col].values)
    t_rmse = rmse(y_valid, gbm.predict(tmp))
    gap = t_rmse-origin  # origin 보다 에러가 커졌으면 이 컬럼은 중요한 것 !
    print(col, t_rmse, gap)
    fin[col] = gap

print(fin)
new = pd.DataFrame()
new['columns'] =fin.keys()
new['values'] = fin.values()
z = pd.concat([z,new])
z.to_csv('featuretest.csv',index=False)


  5%|███▊                                                                               | 1/22 [00:25<08:59, 25.70s/it]

from_lastpoint 2.0587957388800544 0.20881437012845128


  9%|███████▌                                                                           | 2/22 [00:51<08:35, 25.75s/it]

volume_1 2.612689885484157 0.762708516732554


 14%|███████████▎                                                                       | 3/22 [01:17<08:11, 25.88s/it]

...

rstd_28_50 1.86967506187606 0.0196936931244569


 50%|█████████████████████████████████████████                                         | 11/22 [04:44<04:44, 25.88s/it]

rmax_7_7 1.8731459483464679 0.02316457959486473


 55%|████████████████████████████████████████████▋                                     | 12/22 [05:10<04:18, 25.88s/it]

rmax_7_28 1.8611685032288101 0.011187134477206984
